# General Eye localization Dataset Processing pipeline

### Standard Library Imports

In [1]:
import sys, os
import hdf5storage
import numpy as np
import shutil
from statistics import mean
from pathlib import Path


### Local Module Imports

In [2]:
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

from src.utils.paths_manager import LFPW_RAW_SOURCE_PATH, HELEN_RAW_SOURCE_PATH
from src.dataset.loaders import LFPWDataset, HelenDataset, FaceLandmarkDataloaderContext
from src.dataset.ops import FaceLandmarkImageOps
from src.utils.data_structs import ColumnStruct

### Instantiate Main Context class

In [3]:
dloader_ctx = FaceLandmarkDataloaderContext()

In [4]:
# Dataset source container
dataset_container = [
    LFPWDataset(LFPW_RAW_SOURCE_PATH),
    HelenDataset(HELEN_RAW_SOURCE_PATH)
]

# Tset container 
tset_container = [
    'test',
    'train'
]

In [5]:
# Dataset processing settings
DATASETS_PATH = Path('../datasets')

DESTINATION_PATH = DATASETS_PATH / 'processed' / 'MIRRORED'
TEST_DESTINATION_PATH = DESTINATION_PATH / 'test'
TRAIN_DESTINATION_PATH = DESTINATION_PATH / 'train'

if not DESTINATION_PATH.exists():
    DESTINATION_PATH.mkdir()

if not TEST_DESTINATION_PATH.exists():
    TEST_DESTINATION_PATH.mkdir()

if not TRAIN_DESTINATION_PATH.exists():
    TRAIN_DESTINATION_PATH.mkdir()

In [6]:
shutil.copy('../extras/test/see_bboxes.m', str(TEST_DESTINATION_PATH/'see_bboxes.m'))
shutil.copy('../extras/train/see_bboxes.m', str(TRAIN_DESTINATION_PATH/'see_bboxes.m'))

'..\\datasets\\processed\\TRY\\train\\see_bboxes.m'

In [7]:
STANDARD_FACE_WIDTH = 300
STANDARD_FACE_HEIGHT = 420

In [8]:
combined_container = []

for tset in tset_container:
    digitStructPath = DESTINATION_PATH / tset / 'digitStruct.mat'

    if digitStructPath.exists():
        os.remove(digitStructPath)

    for dset in dataset_container:
        dloader_ctx.dloader = dset
        if not dloader_ctx.check_valid():
            print('Invalid Dataset Loader')
            break

        for fl_image in dloader_ctx.load(tset):
            
            orig_fname = os.path.basename(fl_image.fpath)

            # Scale image
            leftmost = min([x[0] for x in fl_image.landmarks.face])
            rightmost = max([x[0] for x in fl_image.landmarks.face])

            face_width = rightmost - leftmost
            scaling_factor = STANDARD_FACE_WIDTH / face_width

            im = fl_image.uniform_scale(scaling_factor)

            # Crop image
            leftmost = min([x[0] for x in im.landmarks.face])
            rightmost = max([x[0] for x in im.landmarks.face])

            face_outline_cluster = im.landmarks.face + im.landmarks.left_brow + im.landmarks.right_brow
            y_median = mean([y[1] for y in face_outline_cluster])

            im = im.crop(left=leftmost,right=rightmost, top=y_median-STANDARD_FACE_HEIGHT/2, bottom=y_median+STANDARD_FACE_HEIGHT/2)

            # Create image with bounding boxes

            bbox_im = im.create_bbox_image()
            bboxed_im = bbox_im.pad_bbox('left_eye', (20, 40, 20, 40)).pad_bbox('right_eye', (20, 40, 20, 40))

            left_eye_square = bboxed_im.get_rect_as_list('left_eye')
            right_eye_square = bboxed_im.get_rect_as_list('right_eye') 

            # print(left_eye_square)
            # print(right_eye_square)
            
            # Create ColumnStruct
            orig_struct = ColumnStruct (filename = orig_fname, header=("filename", "height", "left", "top", "width", "label"), label_by_index=True)
            orig_struct.add(left_eye_square, 1)
            orig_struct.add(right_eye_square, 1)
            combined_container.append(orig_struct.entry())

            bboxed_im.image.save(DESTINATION_PATH / tset / orig_fname)

            # Mirrored 
            mirrored_fname = f"{Path(os.path.basename(fl_image.fpath)).stem}_mirrored{Path(os.path.basename(fl_image.fpath)).suffix}"

            mirrored_im = FaceLandmarkImageOps.mirror(im)

             # Create image with bounding boxes

            mirrored_bbox_im = mirrored_im.create_bbox_image()
            mirrored_bboxed_im = mirrored_bbox_im.pad_bbox('left_eye', (20, 40, 20, 40)).pad_bbox('right_eye', (20, 40, 20, 40))

            mirrored_left_eye_square = mirrored_bboxed_im.get_rect_as_list('left_eye')
            mirrored_right_eye_square = mirrored_bboxed_im.get_rect_as_list('right_eye')

            # print(mirrored_left_eye_square)
            # print(mirrored_right_eye_square)

            # Create ColumnStruct
            mirrored_struct = ColumnStruct (filename = mirrored_fname, header=("filename", "height", "left", "top", "width", "label"), label_by_index=True)
            mirrored_struct.add(mirrored_left_eye_square, 1)
            mirrored_struct.add(mirrored_right_eye_square, 1)
            combined_container.append(mirrored_struct.entry())

            mirrored_bboxed_im.image.save(DESTINATION_PATH / tset / orig_fname)

            # print(os.path.basename(mirrored_fname))

    matrix = {'digitStruct' : np.array([combined_container], dtype=[('name', 'O'), ('bbox', 'O')])}
    hdf5storage.write(matrix, '.', str(DESTINATION_PATH / tset / 'digitStruct.mat') , matlab_compatible=True)